# LAMMPS_ELASTIC Calculation

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2017-07-24

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The LAMMPS_ELASTIC calculation refines the lattice parameters of a structure
and computes the structure's elastic constants, $C_{ij}$, using the ELASTIC script
distributed with the LAMMPS MD software. Energy minimization is used to relax 
the system's box dimensions and local atomic configurations. Once the system 
converges, the elastic constants are estimated by applying small strains. For 
better convergence of the box dimensions, the calculation script runs the 
ELASTIC script multiple times until the box dimensions stop changing.

This calculation provides a quick tool for obtaining both the lattice and
elastic constants for a given structure.

__Disclaimer #1__: With this method there is no guarantee that the resulting
parameters are for a stable structure. The minimization algorithm can have 
trouble relaxing box dimensions if the initial guess is far from the 
equilibrium dimensions. Additionally, some transformation paths may be 
restricted from occurring due to symmetry, i.e. initially cubic structures may 
remain cubic instead of relaxing to a non-cubic structure.

__Disclaimer #2__: The elastic constants are estimated using small strains. 
Depending on the potential, the values for the elastic constants may vary with 
the size of the strain. This can come about either if the strain exceeds the 
linear elastic regime, or if the potential energy is not continuous to the 
fourth derivative. 

- - -

## Method and Theory

The math in this section uses Voigt notation, where indicies i,j correspond to 1=xx, 2=yy, 3=zz, 4=yz, 5=xz, and 6=xy, and x, y and z are orthogonal box vectors.

An initial system (and corresponding unit cell system) is supplied with box dimensions, $a_i^0$, close to the equilibrium values. A LAMMPS simulation performs an energy/force minimization with a fix box_relax option to relax both the local atomic coordinates and the system's dimensions. The pressures, $P_{i}$, of the relaxed state are recorded.

Further energy/force minimizations are then performed without box_relax to evaluate $P_{i}$ at twelve different strain states corresponding to one of $\epsilon_{i}$ being given a value of $\pm \Delta \epsilon$, where $\Delta \epsilon$ is the strain range parameter. Using the $P_{i}$ values obtained from the strained states, the $C_{ij}$ matrix for the system is estimated as

$$ C_{ij} \approx \frac{C_{ij}^+ + C_{ij}^-}{2}, $$

where

$$ C_{ij}^+ = - \frac{P_i(\epsilon_j=\Delta \epsilon) - P_i(\epsilon_j=0)}{\Delta \epsilon}.$$

$$ C_{ij}^- = - \frac{P_i(\epsilon_j=0) - P_i(\epsilon_j=-\Delta \epsilon)}{\Delta \epsilon}.$$

The negative out front comes from the fact that the system-wide stress state is $\sigma_i = -P_i$. 

The full process is repeated until the box dimensions from one iteration to the next are within the specified relative convergence tolerance.

- - -

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import glob
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

### 2. Assign values for the calculation's run parameters

#### 2.0 Set the calculation's working directory

In [2]:
calc_name = 'LAMMPS_ELASTIC'

# Check current working directory
cwd_name = os.path.basename(os.getcwd())

# Change working directory if needed
if cwd_name != calc_name:
    if not os.path.isdir(calc_name):
        os.mkdir(calc_name)
    os.chdir(calc_name)

#### 2.1 Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use (required).

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

- __lib_directory__ defines the relative path to the iprPy library. This makes it easier to define paths to reference records later.

In [3]:
lammps_command = 'lmp_serial'
mpi_command = None
lib_directory = '../../../library'

#### 2.2 Specify the prototype system

- __system__ is an atomman.System to perform the scan on (required). 

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to use for ucell. 

- __prototype_path__ gives the path to the crystal_prototype reference record to use.

- __box_parameters__ defines the initial guess box parameters to scale ucell and system by.

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

In [4]:
prototype_name = 'A1--Cu--fcc'
box_parameters = [3.5, 3.5, 3.5]
sizemults = [3,3,3]

# Define prototype_path using lib_directory and prototype_name
prototype_path = os.path.abspath(os.path.join(lib_directory, 
                                              'crystal_prototype', 
                                              prototype_name+'.json'))

# Create ucell by loading prototype record
ucell = am.load('system_model', prototype_path)[0]
print('# of atoms in ucell =', ucell.natoms)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], 
              b=box_parameters[1], 
              c=box_parameters[2], scale=True)

# Generate system by supersizing ucell
system = am.supersize(ucell, *sizemults)
print('# of atoms in system =', system.natoms)

# of atoms in ucell = 4
# of atoms in system = 108


#### 2.3 Specify the potenital and elemental symbols

- __potential__ is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use (required).

- __symbols__ is a list of the elemental model symbols of potential to associate with the unique atom types of system (required).

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for potential. 

- __potential_path__ gives the path to the potential_LAMMPS reference record to use.

- __potential_dir_path__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file)



In [5]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
symbols = ['Ni']

# Define potential_path and potential_dir_path using lib_directory and potential_name
potential_dir_path = os.path.abspath(os.path.join(lib_directory, 
                                                  'potential_LAMMPS',
                                                  potential_name))
potential_path = potential_dir_path + '.json'

# Create potential by loading LAMMPS-potential record
with open(potential_path) as f:
    potential = lmp.Potential(f, potential_dir_path)
print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.4 Specify calculation-specific run parameters

- __strainrange__ specifies the $\Delta \epsilon$ strain range to use in estimating $C_{ij}$.

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units.

- __pressure_unit__ is the unit of pressure to use for reporting the elastic constants. 

- __convergence_tol__ is the relative tolerance to use in identifying if the lattice constants have converged. 

In [6]:
strainrange = 1e-7
energytolerance = 1e-8
forcetolerance = uc.set_in_units(0.0, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')
pressure_unit = 'GPa'
convergence_tol = 1e-10

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 in.elastic

In [7]:
with open('in.elastic', 'w') as f:
    f.write("""# Compute elastic constant tensor for a crystal
#
# Written by Aidan Thompson (Sandia, athomps@sandia.gov)
#
#  This script uses the following three include files.
#
#   init.mod      (must be modified for different crystal structures)
# 	       	  Define units, deformation parameters and initial
#		  configuration of the atoms and simulation cell.  
#
#
#   potential.mod    (must be modified for different pair styles)
# 		     Define pair style and other attributes 
#		     not stored in restart file
#
#
#   displace.mod    (displace.mod should not need to be modified)
# 		    Perform positive and negative box displacements 
# 		    in direction ${dir} and size ${up}. 
# 		    It uses the resultant changes 
#		    in stress to compute one
# 		    row of the elastic stiffness tensor
#		    
#		    Inputs variables:
#		    	   dir = the Voigt deformation component 
#		    		    (1,2,3,4,5,6)  
#		    Global constants:
#       	    	   up = the deformation magnitude (strain units)
#       		   cfac = conversion from LAMMPS pressure units to 
#               	   output units for elastic constants 
#
#
#  To run this on a different system, it should only be necessary to 
#  modify the files init.mod and potential.mod. In order to calculate
#  the elastic constants correctly, care must be taken to specify
#  the correct units in init.mod (units, cfac and cunits). It is also
#  important to verify that the minimization of energy w.r.t atom
#  positions in the deformed cell is fully converged.
#  One indication of this is that the elastic constants are insensitive
#  to the choice of the variable ${up} in init.mod. Another is to check
#  the final max and two-norm forces reported in the log file. If you know
#  that minimization is not required, you can set maxiter = 0.0 in 
#  init.mod. 
#
#  There are two alternate versions of displace.mod provided.
#  They are displace_restart.mod and displace_reverse.mod. 
#  The former resets the box using a restart file while 
#  the latter reverses the deformation. Copy whichever
#  one you like best to displace.mod.
#

include init.mod
include potential.mod

# Compute initial state
fix 3 all box/relax  aniso 0.0
dump config all custom ${maxiter} atom.* id type x y z
minimize ${etol} ${ftol} ${maxiter} ${maxeval}
undump config

variable tmp equal pxx
variable pxx0 equal ${tmp}
variable tmp equal pyy
variable pyy0 equal ${tmp}
variable tmp equal pzz
variable pzz0 equal ${tmp}
variable tmp equal pyz
variable pyz0 equal ${tmp}
variable tmp equal pxz
variable pxz0 equal ${tmp}
variable tmp equal pxy
variable pxy0 equal ${tmp}

variable tmp equal lx
variable lx0 equal ${tmp}
variable tmp equal ly
variable ly0 equal ${tmp}
variable tmp equal lz
variable lz0 equal ${tmp}

# These formulas define the derivatives w.r.t. strain components
# Constants uses $, variables use v_ 
variable d1 equal -(v_pxx1-${pxx0})/(v_delta/v_len0)*${cfac}
variable d2 equal -(v_pyy1-${pyy0})/(v_delta/v_len0)*${cfac}
variable d3 equal -(v_pzz1-${pzz0})/(v_delta/v_len0)*${cfac}
variable d4 equal -(v_pyz1-${pyz0})/(v_delta/v_len0)*${cfac}
variable d5 equal -(v_pxz1-${pxz0})/(v_delta/v_len0)*${cfac}
variable d6 equal -(v_pxy1-${pxy0})/(v_delta/v_len0)*${cfac}

# Write restart
unfix 3
write_restart restart.equil

# uxx Perturbation

variable dir equal 1
include displace.mod

# uyy Perturbation

variable dir equal 2
include displace.mod

# uzz Perturbation

variable dir equal 3
include displace.mod

# uyz Perturbation

variable dir equal 4
include displace.mod

# uxz Perturbation

variable dir equal 5
include displace.mod

# uxy Perturbation

variable dir equal 6
include displace.mod

# Output final values

variable C11all equal ${C11}
variable C22all equal ${C22}
variable C33all equal ${C33}

variable C12all equal 0.5*(${C12}+${C21})
variable C13all equal 0.5*(${C13}+${C31})
variable C23all equal 0.5*(${C23}+${C32})

variable C44all equal ${C44}
variable C55all equal ${C55}
variable C66all equal ${C66}

variable C14all equal 0.5*(${C14}+${C41})
variable C15all equal 0.5*(${C15}+${C51})
variable C16all equal 0.5*(${C16}+${C61})

variable C24all equal 0.5*(${C24}+${C42})
variable C25all equal 0.5*(${C25}+${C52})
variable C26all equal 0.5*(${C26}+${C62})

variable C34all equal 0.5*(${C34}+${C43})
variable C35all equal 0.5*(${C35}+${C53})
variable C36all equal 0.5*(${C36}+${C63})

variable C45all equal 0.5*(${C45}+${C54})
variable C46all equal 0.5*(${C46}+${C64})
variable C56all equal 0.5*(${C56}+${C65})

print "Elastic Constant C11all = ${C11all} ${cunits}"
print "Elastic Constant C22all = ${C22all} ${cunits}"
print "Elastic Constant C33all = ${C33all} ${cunits}"

print "Elastic Constant C12all = ${C12all} ${cunits}"
print "Elastic Constant C13all = ${C13all} ${cunits}"
print "Elastic Constant C23all = ${C23all} ${cunits}"

print "Elastic Constant C44all = ${C44all} ${cunits}"
print "Elastic Constant C55all = ${C55all} ${cunits}"
print "Elastic Constant C66all = ${C66all} ${cunits}"

print "Elastic Constant C14all = ${C14all} ${cunits}"
print "Elastic Constant C15all = ${C15all} ${cunits}"
print "Elastic Constant C16all = ${C16all} ${cunits}"

print "Elastic Constant C24all = ${C24all} ${cunits}"
print "Elastic Constant C25all = ${C25all} ${cunits}"
print "Elastic Constant C26all = ${C26all} ${cunits}"

print "Elastic Constant C34all = ${C34all} ${cunits}"
print "Elastic Constant C35all = ${C35all} ${cunits}"
print "Elastic Constant C36all = ${C36all} ${cunits}"

print "Elastic Constant C45all = ${C45all} ${cunits}"
print "Elastic Constant C46all = ${C46all} ${cunits}"
print "Elastic Constant C56all = ${C56all} ${cunits}"
""")

#### 3.2 displace.mod

In [8]:
with open('displace.mod', 'w') as f:
    f.write("""# NOTE: This script should not need to be
# modified. See in.elastic for more info.
#
# Find which reference length to use

if "${dir} == 1" then &
   "variable len0 equal ${lx0}" 
if "${dir} == 2" then &
   "variable len0 equal ${ly0}" 
if "${dir} == 3" then &
   "variable len0 equal ${lz0}" 
if "${dir} == 4" then &
   "variable len0 equal ${lz0}" 
if "${dir} == 5" then &
   "variable len0 equal ${lz0}" 
if "${dir} == 6" then &
   "variable len0 equal ${ly0}" 

# Reset box and simulation parameters

clear
read_restart restart.equil
include potential.mod

# Negative deformation

variable delta equal -${up}*${len0}
if "${dir} == 1" then &
   "change_box all x delta 0 ${delta} remap units box"
if "${dir} == 2" then &
   "change_box all y delta 0 ${delta} remap units box"
if "${dir} == 3" then &
   "change_box all z delta 0 ${delta} remap units box"
if "${dir} == 4" then &
   "change_box all yz delta ${delta} remap units box"
if "${dir} == 5" then &
   "change_box all xz delta ${delta} remap units box"
if "${dir} == 6" then &
   "change_box all xy delta ${delta} remap units box"

# Relax atoms positions

minimize ${etol} ${ftol} ${maxiter} ${maxeval}

# Obtain new stress tensor
 
variable tmp equal pxx
variable pxx1 equal ${tmp}
variable tmp equal pyy
variable pyy1 equal ${tmp}
variable tmp equal pzz
variable pzz1 equal ${tmp}
variable tmp equal pxy
variable pxy1 equal ${tmp}
variable tmp equal pxz
variable pxz1 equal ${tmp}
variable tmp equal pyz
variable pyz1 equal ${tmp}

# Compute elastic constant from pressure tensor

variable C1neg equal ${d1}
variable C2neg equal ${d2}
variable C3neg equal ${d3}
variable C4neg equal ${d4}
variable C5neg equal ${d5}
variable C6neg equal ${d6}

# Reset box and simulation parameters

clear
read_restart restart.equil
include potential.mod

# Positive deformation

variable delta equal ${up}*${len0}
if "${dir} == 1" then &
   "change_box all x delta 0 ${delta} remap units box"
if "${dir} == 2" then &
   "change_box all y delta 0 ${delta} remap units box"
if "${dir} == 3" then &
   "change_box all z delta 0 ${delta} remap units box"
if "${dir} == 4" then &
   "change_box all yz delta ${delta} remap units box"
if "${dir} == 5" then &
   "change_box all xz delta ${delta} remap units box"
if "${dir} == 6" then &
   "change_box all xy delta ${delta} remap units box"

# Relax atoms positions

minimize ${etol} ${ftol} ${maxiter} ${maxeval}

# Obtain new stress tensor
 
variable tmp equal pe
variable e1 equal ${tmp}
variable tmp equal press
variable p1 equal ${tmp}
variable tmp equal pxx
variable pxx1 equal ${tmp}
variable tmp equal pyy
variable pyy1 equal ${tmp}
variable tmp equal pzz
variable pzz1 equal ${tmp}
variable tmp equal pxy
variable pxy1 equal ${tmp}
variable tmp equal pxz
variable pxz1 equal ${tmp}
variable tmp equal pyz
variable pyz1 equal ${tmp}

# Compute elastic constant from pressure tensor

variable C1pos equal ${d1}
variable C2pos equal ${d2}
variable C3pos equal ${d3}
variable C4pos equal ${d4}
variable C5pos equal ${d5}
variable C6pos equal ${d6}

# Combine positive and negative 

variable C1${dir} equal 0.5*(${C1neg}+${C1pos})
variable C2${dir} equal 0.5*(${C2neg}+${C2pos})
variable C3${dir} equal 0.5*(${C3neg}+${C3pos})
variable C4${dir} equal 0.5*(${C4neg}+${C4pos})
variable C5${dir} equal 0.5*(${C5neg}+${C5pos})
variable C6${dir} equal 0.5*(${C6neg}+${C6pos})

# Delete dir to make sure it is not reused

variable dir delete
""")

#### 3.3 init.mod.template

In [9]:
with open('init.mod.template', 'w') as f:
    f.write("""# NOTE: This script can be modified for different atomic structures, 
# units, etc. See in.elastic for more info.
#

# Define the finite deformation size. Try several values of this
# variable to verify that results do not depend on it.
variable up equal <strainrange>
 
# pressure unit scaling factor and string
variable cfac equal 1.0e-4
variable cunits string GPa

# Define minimization parameters
variable etol equal <etol>
variable ftol equal <ftol>
variable maxiter equal <maxiter>
variable maxeval equal <maxeval>
variable dmax equal <dmax>

# generate system
<atomman_system_info>

change_box all triclinic
""")

#### 3.4 potential.mod.template

In [10]:
with open('potential.mod.template', 'w') as f:
    f.write("""# NOTE: This script can be modified for different pair styles 
# See in.elastic for more info.

# Choose potential
<atomman_pair_info>

# Setup neighbor style
neighbor 1.0 nsq
neigh_modify once no every 1 delay 0 check yes

# Setup minimization style
min_style	     cg
min_modify	     dmax ${dmax} line quadratic

# Setup output
thermo		1
thermo_style custom step temp pe press pxx pyy pzz pxy pxz pyz lx ly lz vol
thermo_modify norm no
""")

#### 3.5 lammps_ELASTIC_refine()

In [11]:
def lammps_ELASTIC_refine(lammps_command, system, potential, symbols,
                          mpi_command=None, ucell=None,
                          strainrange=1e-6, etol=0.0, ftol=0.0, maxiter=10000,
                          maxeval=100000, dmax=uc.set_in_units(0.01, 'angstrom'),
                          pressure_unit='GPa', tol=1e-10):
    """
    Repeatedly runs the ELASTIC example distributed with LAMMPS until box
    dimensions converge within a tolerance.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The system to perform the calculation on.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    symbols : list of str
        The list of element-model symbols for the Potential that correspond to
        system's atypes.
    mpi_command : str, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    ucell : atomman.System, optional
        The fundamental unit cell correspodning to system.  This is used to
        convert system dimensions to cell dimensions. If not given, ucell will
        be taken as system.
    strainrange : float, optional
        The small strain value to apply when calculating the elastic
        constants (default is 1e-6).
    etol : float, optional
        The energy tolerance for the structure minimization. This value is
        unitless. (Default is 0.0).
    ftol : float, optional
        The force tolerance for the structure minimization. This value is in
        units of force. (Default is 0.0).
    maxiter : int, optional
        The maximum number of minimization iterations to use (default is 10000).
    maxeval : int, optional
        The maximum number of minimization evaluations to use (default is 
        100000).
    dmax : float, optional
        The maximum distance in length units that any atom is allowed to relax
        in any direction during a single minimization iteration (default is
        0.01 Angstroms).
    pressure_unit : str, optional
        The unit of pressure to calculate the elastic constants in (default is
        'GPa').
    tol : float, optional
        The relative tolerance used to determine if the lattice constants have
        converged (default is 1e-10).
    
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'a_lat'** (*float*) - The relaxed a lattice constant.
        - **'b_lat'** (*float*) - The relaxed b lattice constant.
        - **'c_lat'** (*float*) - The relaxed c lattice constant.
        - **'alpha_lat'** (*float*) - The alpha lattice angle.
        - **'beta_lat'** (*float*) - The beta lattice angle.
        - **'gamma_lat'** (*float*) - The gamma lattice angle.
        - **'E_coh'** (*float*) - The cohesive energy of the relaxed system.
        - **'stress'** (*numpy.array*) - The measured stress state of the
          relaxed system.
        - **'C_elastic'** (*atomman.ElasticConstants*) - The relaxed system's
          elastic constants.
        - **'system_relaxed'** (*atomman.System*) - The relaxed system.
    """
    
    # Set ucell = system if ucell not given
    if ucell is None:
        ucell = system
    
    # Get ratios of lx, ly, and lz of system relative to a of ucell
    lx_a = system.box.a / ucell.box.a
    ly_b = system.box.b / ucell.box.b
    lz_c = system.box.c / ucell.box.c
    alpha = system.box.alpha
    beta =  system.box.beta
    gamma = system.box.gamma
    
    old_results = lammps_ELASTIC(lammps_command, system, potential, symbols,
                                 mpi_command=mpi_command, ucell=None,
                                 strainrange=strainrange,
                                 etol=etol, ftol=ftol,
                                 maxiter=maxiter, maxeval=maxeval,
                                 dmax=dmax, pressure_unit=pressure_unit)
    shutil.move('log.lammps', 'elastic-0-log.lammps')
    os.remove('atom.0')
    for atom_dump in glob.iglob('atom.*'):
        shutil.move(atom_dump, 'elastic-0-config.dump')
    
    converged = False
    for cycle in xrange(1, 100):
        new_results = lammps_ELASTIC(lammps_command,
                                     old_results['system_relaxed'],
                                     potential, symbols, 
                                     mpi_command=mpi_command,
                                     ucell=None,
                                     strainrange=strainrange,
                                     etol=etol, ftol=ftol, 
                                     maxiter=maxiter, maxeval=maxeval,
                                     dmax=dmax, pressure_unit=pressure_unit)
        shutil.move('log.lammps', 'elastic-'+str(cycle)+'-log.lammps')
        os.remove('atom.0')
        for atom_dump in glob.iglob('atom.*'):
            shutil.move(atom_dump, 'elastic-'+str(cycle)+'-config.dump')
        
        # Test if box dimensions have converged
        if (np.isclose(old_results['a_lat'],
                       new_results['a_lat'],
                       rtol=tol, atol=0)
          and
            np.isclose(old_results['b_lat'],
                       new_results['b_lat'],
                       rtol=tol, atol=0)
          and
            np.isclose(old_results['c_lat'],
                       new_results['c_lat'],
                       rtol=tol, atol=0)):
            converged = True
            break
        else:
            old_results = new_results
    
    # Return values if converged
    if converged:
        # Scale lattice constants to ucell 
        new_results['a_lat'] = new_results['a_lat'] / lx_a
        new_results['b_lat'] = new_results['b_lat'] / ly_b
        new_results['c_lat'] = new_results['c_lat'] / lz_c
        
        return new_results
    else:
        raise RuntimeError('Failed to converge after 100 cycles')
        


#### 3.6 lammps_ELASTIC()

In [12]:
def lammps_ELASTIC(lammps_command, system, potential, symbols,
                   mpi_command=None, ucell=None,
                   strainrange=1e-6, etol=0.0, ftol=0.0, maxiter=10000,
                   maxeval=100000, dmax=uc.set_in_units(0.01, 'angstrom'),
                   pressure_unit='GPa'):
    """
    Sets up and runs the ELASTIC example distributed with LAMMPS.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The system to perform the calculation on.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    symbols : list of str
        The list of element-model symbols for the Potential that correspond to
        system's atypes.
    mpi_command : str, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    ucell : atomman.System, optional
        The fundamental unit cell correspodning to system.  This is used to
        convert system dimensions to cell dimensions. If not given, ucell will
        be taken as system.
    strainrange : float, optional
        The small strain value to apply when calculating the elastic
        constants (default is 1e-6).
    etol : float, optional
        The energy tolerance for the structure minimization. This value is
        unitless. (Default is 0.0).
    ftol : float, optional
        The force tolerance for the structure minimization. This value is in
        units of force. (Default is 0.0).
    maxiter : int, optional
        The maximum number of minimization iterations to use (default is 10000).
    maxeval : int, optional
        The maximum number of minimization evaluations to use (default is 
        100000).
    dmax : float, optional
        The maximum distance in length units that any atom is allowed to relax
        in any direction during a single minimization iteration (default is 
        0.01 Angstroms).
    pressure_unit : str, optional
        The unit of pressure to calculate the elastic constants in (default is
        'GPa').
    
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'a_lat'** (*float*) - The relaxed a lattice constant.
        - **'b_lat'** (*float*) - The relaxed b lattice constant.
        - **'c_lat'** (*float*) - The relaxed c lattice constant.
        - **'alpha_lat'** (*float*) - The alpha lattice angle.
        - **'beta_lat'** (*float*) - The beta lattice angle.
        - **'gamma_lat'** (*float*) - The gamma lattice angle.
        - **'E_coh'** (*float*) - The cohesive energy of the relaxed system.
        - **'stress'** (*numpy.array*) - The measured stress state of the
          relaxed system.
        - **'C_elastic'** (*atomman.ElasticConstants*) - The relaxed system's
          elastic constants.
        - **'system_relaxed'** (*atomman.System*) - The relaxed system.
    """

    # Set ucell = system if ucell not given
    if ucell is None:
        ucell = system
    
    # Get ratios of lx, ly, and lz of system relative to a of ucell
    lx_a = system.box.a / ucell.box.a
    ly_b = system.box.b / ucell.box.b
    lz_c = system.box.c / ucell.box.c
    alpha = system.box.alpha
    beta =  system.box.beta
    gamma = system.box.gamma
    
    # Get lammps units
    lammps_units = lmp.style.unit(potential.units)

    # Define lammps variables
    lammps_variables = {}
    system_info = lmp.atom_data.dump(system, 'init.dat',
                                     units=potential.units,
                                     atom_style=potential.atom_style)
    lammps_variables['atomman_system_info'] = system_info
    lammps_variables['atomman_pair_info'] = potential.pair_info(symbols)
    lammps_variables['strainrange'] = strainrange
    pressure_scale = uc.get_in_units(uc.set_in_units(1.0,
                                                    lammps_units['pressure']),
                                     pressure_unit)
    lammps_variables['pressure_unit_scaling'] = pressure_scale
    lammps_variables['pressure_unit'] = pressure_unit
    lammps_variables['etol'] = etol
    lammps_variables['ftol'] = uc.get_in_units(ftol, lammps_units['force'])
    lammps_variables['maxiter'] = maxiter
    lammps_variables['maxeval'] = maxeval
    lammps_variables['dmax'] = uc.get_in_units(dmax, lammps_units['length'])
    
    # Fill in mod.template files
    with open('init.mod.template') as template_file:
        template = template_file.read()
    with open('init.mod', 'w') as in_file:
        in_file.write(iprPy.tools.filltemplate(template, lammps_variables,
                                               '<', '>'))
    with open('potential.mod.template') as template_file:
        template = template_file.read()
    with open('potential.mod', 'w') as in_file:
        in_file.write(iprPy.tools.filltemplate(template, lammps_variables,
                                               '<', '>'))
    
    output = lmp.run(lammps_command, 'in.elastic', mpi_command)
    
    # Extract output values
    thermo = output.simulations[0]['thermo']
    
    results = {}
    results['E_coh'] = uc.set_in_units(thermo.PotEng.values[-1] / system.natoms,
                                       lammps_units['energy'])
    results['a_lat'] = uc.set_in_units(thermo.Lx.values[-1] / lx_a,
                                       lammps_units['length'])
    results['b_lat'] = uc.set_in_units(thermo.Ly.values[-1] / ly_b,
                                       lammps_units['length'])
    results['c_lat'] = uc.set_in_units(thermo.Lz.values[-1] / lz_c,
                                       lammps_units['length'])
    results['alpha_lat'] = alpha
    results['beta_lat'] =  beta
    results['gamma_lat'] = gamma
    
    pxx = uc.set_in_units(thermo.Pxx.values[-1], lammps_units['pressure'])
    pyy = uc.set_in_units(thermo.Pyy.values[-1], lammps_units['pressure'])
    pzz = uc.set_in_units(thermo.Pzz.values[-1], lammps_units['pressure'])
    pxy = uc.set_in_units(thermo.Pxy.values[-1], lammps_units['pressure'])
    pxz = uc.set_in_units(thermo.Pxz.values[-1], lammps_units['pressure'])
    pyz = uc.set_in_units(thermo.Pyz.values[-1], lammps_units['pressure'])
    results['stress'] = -1 * np.array([[pxx, pxy, pxz],
                                       [pxy, pyy, pyz],
                                       [pxz, pyz, pzz]])
    
    # Load relaxed system from dump file
    last_dump_file = 'atom.'+str(thermo.Step.values[-1])
    results['system_relaxed'] = lmp.atom_dump.load(last_dump_file)
    
    with open('log.lammps') as log_file:
        log = log_file.read()
    
    start = log.find('print "Elastic Constant C11all = ${C11all} ${cunits}"')
    lines = log[start+54:].split('\n')

    Cdict = {}
    for line in lines:
        terms = line.split()
        if len(terms) > 0 and terms[0] == 'Elastic':
            c_term = terms[2][:3]
            c_value = float(terms[4])
            c_unit = terms[5]
            
            Cdict[c_term] = uc.set_in_units(c_value, c_unit)
    results['C_elastic'] = am.ElasticConstants(**Cdict)
                
    return results



### 4. Run calculation function(s)

In [13]:
results_dict = lammps_ELASTIC_refine(lammps_command, 
                                     system, 
                                     potential,
                                     symbols,
                                     mpi_command = mpi_command,
                                     ucell=ucell,
                                     etol=1e-8,
                                     strainrange = strainrange,
                                     maxiter=1000,
                                     maxeval=10000)

In [14]:
results_dict.keys()

['stress',
 'gamma_lat',
 'a_lat',
 'beta_lat',
 'b_lat',
 'alpha_lat',
 'c_lat',
 'C_elastic',
 'E_coh',
 'system_relaxed']

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display relaxed lattice constants in.
- __energy_unit__ is the unit of energy to display cohesive energies in.
- __pressure_unit__ is the unit of pressure to display Cij and relaxed stress state in.

In [15]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'

#### 5.2 Print Ecoh and lattice constants of relaxed ucell

In [16]:
print('Ecoh =', uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)
print('a =',    uc.get_in_units(results_dict['a_lat'], length_unit), length_unit)
print('b =',    uc.get_in_units(results_dict['b_lat'], length_unit), length_unit)
print('c =',    uc.get_in_units(results_dict['c_lat'], length_unit), length_unit)
print('alpha =', results_dict['alpha_lat'])
print('beta = ', results_dict['beta_lat'])
print('gamma =', results_dict['gamma_lat'])

Ecoh = -4.45 eV
a = 3.51999933333 angstrom
b = 3.51999933333 angstrom
c = 3.51999933333 angstrom
alpha = 90.0
beta =  90.0
gamma = 90.0


#### 5.3 Print Cij

In [17]:
print('Cij ('+pressure_unit+') =')
for Ci in uc.get_in_units(results_dict['C_elastic'].Cij, pressure_unit):
    print('[%9.4f %9.4f %9.4f %9.4f %9.4f %9.4f]' % tuple(Ci))

Cij (GPa) =
[ 247.8624  147.8284  147.8284    0.0000    0.0000    0.0000]
[ 147.8284  247.8624  147.8284    0.0000    0.0000    0.0000]
[ 147.8284  147.8284  247.8624    0.0000    0.0000    0.0000]
[   0.0000    0.0000    0.0000  124.8381    0.0000    0.0000]
[   0.0000    0.0000    0.0000    0.0000  124.8381    0.0000]
[   0.0000    0.0000    0.0000    0.0000    0.0000  124.8381]


#### 5.4 Check stress state of relaxed system

In [18]:
print('stress ('+pressure_unit+') =')
for Si in uc.get_in_units(results_dict['stress'], pressure_unit):
    print('[%9.4f %9.4f %9.4f]' % tuple(Si))

stress (GPa) =
[  -0.0000    0.0000    0.0000]
[   0.0000   -0.0000    0.0000]
[   0.0000    0.0000   -0.0000]


#### 5.5 Show relaxed atomic configuration

In [19]:
print(results_dict['system_relaxed'])

avect =  [10.560,  0.000,  0.000]
bvect =  [ 0.000, 10.560,  0.000]
cvect =  [ 0.000,  0.000, 10.560]
origin = [-0.030, -0.030, -0.030]
natoms = 108
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |  -0.030 |  -0.030 |  -0.030
      1 |       1 |  -0.030 |   1.730 |   1.730
      2 |       1 |   1.730 |  -0.030 |   1.730
      3 |       1 |   1.730 |   1.730 |  -0.030
      4 |       1 |   3.490 |  -0.030 |  -0.030
      5 |       1 |   3.490 |   1.730 |   1.730
      6 |       1 |   5.250 |  -0.030 |   1.730
      7 |       1 |   5.250 |   1.730 |  -0.030
      8 |       1 |   7.010 |  -0.030 |  -0.030
      9 |       1 |   7.010 |   1.730 |   1.730
     10 |       1 |   8.770 |  -0.030 |   1.730
     11 |       1 |   8.770 |   1.730 |  -0.030
     12 |       1 |  -0.030 |   3.490 |  -0.030
     13 |       1 |  -0.030 |   5.250 |   1.730
     14 |       1 |   1.730 |   3.490 |   1.730
     15 |       1 |   1.730 |   5.250 |  -0.030
     16 |       1 |   3